In [1]:
import random
from util_class.ImageStruct import ImageStruct
import heapq
import torch

In [2]:
annotation_file, tensor_file = \
      './data/anotation_new.csv', './image_tensor/model_image_2023-06-28/resnet50_tensor.pt'
images = ImageStruct(annotation_file, tensor_file)

In [18]:
src_index = random.randint(0, len(images))
s_img_tensor, s_img_path, s_category, s_caption = images.get(src_index)
s_category, s_caption

('レッグウェア × 灰色（グレー）',
 ' ベーシックなソリッドタイツ。トレンドに左右されない秋冬のマストアイテムです。股上深めのシルエットなので、腰もしっかりあたためてくれます。')

In [4]:
import torchvision.transforms as transforms
from PIL import Image
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn

if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得
"""
テキスト処理のモデル
"""
class CaptionEncoder(nn.Module):
  def __init__(self):
    super().__init__()
    self.bert = AutoModel.from_pretrained("cl-tohoku/bert-base-japanese-v2")
  def forward(self, x):
    x = self.bert(x)
    x = torch.max(x.last_hidden_state, dim=1)[0]  # max pooling
    return x

model_name =  './model/model_caption_2023-06-28.pth'
caption_model = CaptionEncoder().to(device)
caption_model.load_state_dict(torch.load(model_name))
caption_model.eval()

tokenizer = AutoTokenizer.from_pretrained("cl-tohoku/bert-base-japanese-v2")


Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
ids = tokenizer.encode("冬、寒い、フォーマル、アウター、上品、高級", return_tensors='pt')
ids = ids.to(device)

In [20]:
s_caption_tensor = caption_model(ids)[0].to('cpu')

In [7]:
from util.category import get_item_id

In [21]:

heap = []
MAX = 100000
for i in range(MAX):
    if (i % 1000 == 0):
        print(f"{i * 100 / MAX} %")
    # img_tensor, img_path, category
    t_img_tensor, t_img_path, t_category, _ = images.get(i)
    t_id = get_item_id(t_img_path)
    heap_ids = [get_item_id(i[1]) for i in heap]
    if (t_id in heap_ids):
        continue
    # print(s_category, t_category)
    # if (s_category != t_category):
    #     continue
    dist = torch.dist(s_caption_tensor, t_img_tensor, p=2)
    heapq.heappush(heap, (-dist.item(), t_img_path))
    if (len(heap) > 10):
        heapq.heappop(heap)
    
for i in range(len(heap)):
    print(heap[i])

0.0 %
1.0 %
2.0 %
3.0 %
4.0 %
5.0 %
6.0 %
7.0 %
8.0 %
9.0 %
10.0 %
11.0 %
12.0 %
13.0 %
14.0 %
15.0 %
16.0 %
17.0 %
18.0 %
19.0 %
20.0 %
21.0 %
22.0 %
23.0 %
24.0 %
25.0 %
26.0 %
27.0 %
28.0 %
29.0 %
30.0 %
31.0 %
32.0 %
33.0 %
34.0 %
35.0 %
36.0 %
37.0 %
38.0 %
39.0 %
40.0 %
41.0 %
42.0 %
43.0 %
44.0 %
45.0 %
46.0 %
47.0 %
48.0 %
49.0 %
50.0 %
51.0 %
52.0 %
53.0 %
54.0 %
55.0 %
56.0 %
57.0 %
58.0 %
59.0 %
60.0 %
61.0 %
62.0 %
63.0 %
64.0 %
65.0 %
66.0 %
67.0 %
68.0 %
69.0 %
70.0 %
71.0 %
72.0 %
73.0 %
74.0 %
75.0 %
76.0 %
77.0 %
78.0 %
79.0 %
80.0 %
81.0 %
82.0 %
83.0 %
84.0 %
85.0 %
86.0 %
87.0 %
88.0 %
89.0 %
90.0 %
91.0 %
92.0 %
93.0 %
94.0 %
95.0 %
96.0 %
97.0 %
98.0 %
99.0 %
(-18.912038803100586, 'D:/M1/fashion/IQON/IQON3000\\1110748\\1834204/4789523_m.jpg')
(-18.909568786621094, 'D:/M1/fashion/IQON/IQON3000\\1029943\\3021602/8422091_m.jpg')
(-18.90789222717285, 'D:/M1/fashion/IQON/IQON3000\\1110748\\2141117/5150505_m.jpg')
(-18.907625198364258, 'D:/M1/fashion/IQON/IQON3000\\1011

In [22]:
from util.save_combined_image import save_combined_image


save_combined_image(s_img_path, [i[1] for i in heap], f'./result/caption/test4.png')